## Overview
## Outline
1. Import Library
2. Setup pre-requisites
3. Extract dataset to images
4. Upload images to Azure data store
4. Setting up Azure ML Infrastructure

### 1. Import library

In [1]:
## Install needed packages
%pip install opencv-python torch torchvision torchaudio azureml-core

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 713.3 kB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.8/614.8 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.8 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 

In [3]:
import os
from importlib import reload

import cv2
# from azureml.core import Workspace, Dataset, Datastore

## Using torchvision to create a dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision
import torch


### 2. Setup pre-requisites

Clone source code from github

In [4]:
## curl source code from github
!wget 'https://github.com/nnvu-fit/iusai-project/archive/refs/heads/main.zip' -O main.zip
!unzip main.zip
!mv iusai-project-main/* .
!rm -rf iusai-project-main main.zip

--2024-07-22 19:33:20--  https://github.com/nnvu-fit/iusai-project/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nnvu-fit/iusai-project/zip/refs/heads/main [following]
--2024-07-22 19:33:21--  https://codeload.github.com/nnvu-fit/iusai-project/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [  <=>               ] 307,35K  1,46MB/s    in 0,2s    

2024-07-22 19:33:21 (1,46 MB/s) - ‘main.zip’ saved [314727]

Archive:  main.zip
30c6d623a5535529b640fc32926bb83022b6ba2c
   creating: iusai-project-main/
   creating: iusai-project-main/.devcon

In [3]:
## Setup Azure ML Workspace
ws = Workspace.from_config()
## From workspace, get/create the default datastore
ds = ws.get_default_datastore()
ws, ds

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


UserErrorException: UserErrorException:
	Message: You are currently logged-in to 66bbbbf2-d789-4231-8e48-08bfeb9cba42 tenant. You don't have access to d554f489-6933-4c33-8722-a536b3682bd7 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='DEKRA_Application_Subscription', subscription_id='8db25668-388f-4e79-883a-54133883a597')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to 66bbbbf2-d789-4231-8e48-08bfeb9cba42 tenant. You don't have access to d554f489-6933-4c33-8722-a536b3682bd7 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='DEKRA_Application_Subscription', subscription_id='8db25668-388f-

### 3. Extract dataset to images

In [1]:
## define videos location + images output location
video_path = 'videos/'
images_path = 'subjects-small/'
print('video_path: ', video_path)
print('images_path: ', images_path)

video_path:  videos/
images_path:  subjects-small/


In [4]:
import v2i
reload(v2i)
from v2i import extract_images_from_videos
import threading

def extract_images_from_videos_collection(video_path, images_path):
    ## get all videos file in video_path
    video_list_location_collection = os.listdir(video_path)

    ## define total_label_dict
    ## create threading pool
    thread = []
    ## for each video file
    for video_list_location in video_list_location_collection:
        ## check if video_location is not a directory (i.e. is a file), then skip
        if not os.path.isdir(video_path + video_list_location):
            continue
        ## list videos in video_location
        thread.append(threading.Thread(target=extract_images_from_videos, args=(video_path + video_list_location, images_path, 1)))
        # extract_images_from_videos(video_path + video_list_location, images_path, inteval=1)

    ## start all threads
    for t in thread: t.start()
    ## wait for all threads to finish
    for t in thread:
        t.join()
    
    ## check images in images_path
    images_list_location_collection = os.listdir(images_path)
    print('images_list_location_collection: ', images_list_location_collection)

# video_subject_6_path = video_path + '/subject6-###'
# ## list videos in video_location
# label_dict = extract_images_from_videos(video_subject_6_path, images_path)

## extract images from videos
extract_images_from_videos_collection(video_path, images_path)


Label dict of videos/subject11/alhashe31.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject13/guolingh1.avi (start): [['0', 0], ['6', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject10/huangpi21.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject12/makiluke1.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject15/bajajpak1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject14/chuangy61.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject1/Yousef1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['5', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject16/dhingra51.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject17/zhoulian1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject21/Mark1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject2/Jourabloo1.avi (start): [['0', 0], ['6', 0], ['1', 0], ['2', 0], ['3', 0]]
Label dict of videos/subject19/w

### 4. Upload images to Azure data store

In [ ]:
## upload images to data asset using Dataset.File.upload_directory
Dataset.File.upload_directory(src_dir=images_path, target=(ds, 'images-extra-small'), overwrite=True, show_progress=True) \
    .register(workspace=ws, name='images-extra-small', description='images-extra-small') ## register dataset \\


Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from 'c:/Users/nnvuf/source/ai-final-project/images' to 'images-extra-small'
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/images-extra-small')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "22c174ee-e3e7-4e60-a949-e20eb237c73d",
    "name": "images-extra-small",
    "version": 1,
    "description": "images-extra-small",
    "workspace": "Workspace.create(name='ws-vunn-iusai-sea-sp6k7', subscription_id='d554f489-6933-4c33-8722-a536b3682bd7', resource_group='rg-vunn-iusai-sea-sp6k7')"
  }
}

### 5. Setup public workspace endpoint

In [ ]:
dataset_collection_list = ['images-extra-small', 'images-small-v2', 'images-medium']
## register dataset using Dataset.File.from_files
for dataset_collection in dataset_collection_list:
    ## check if dataset_collection is already registered
    if dataset_collection in ws.datasets.keys():
        print('dataset_collection: ', dataset_collection, ' is already registered')
        continue
    Dataset.File.from_files(path=(ds, dataset_collection)) \
        .register(workspace=ws, name=dataset_collection, description=dataset_collection) ## register dataset \\


dataset_collection:  images-extra-small  is already registered
dataset_collection:  images-small-v2  is already registered
dataset_collection:  images-medium  is already registered


## Define and Train models
1. ResNET
2. DenseNET

### Setup device + load dataset

In [5]:
## import train and scrore function from train.py
import train as t;
reload(t)
from train import cross_validate, score_model, get_device

device = get_device()
device

'cuda'

In [7]:
images_path = 'images'
images_ds_path = "images-extra-small"
if not os.path.exists(images_path):
    os.mkdir(images_path)
# download data asset to local if images_path is empty
if len(os.listdir(images_path)) == 0:
    print('images_path is empty, download images_ds to images_path')
    # download data asset to local
    images_ds = Dataset.get_by_name(workspace=ws, name=images_ds_path)
    images_ds.download(target_path=images_path, overwrite=True)

images_path is empty, download images_ds to images_path


NameError: name 'Dataset' is not defined

#### Prepare Dataset

In [8]:
from torch.utils.data import DataLoader
import data_set as ds
reload(ds)

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
imageDataset = ds.ImageDataset('images-small', transform=transform)
## define batch_size
batch_size = 32

In [9]:
image, label = imageDataset.get_image(0)
# image.show()
labels = imageDataset.labels()
## show labels in Interger
print('labels: ',  [int(l) for l in labels])

labels:  [0, 1, 2, 3, 5, 6]


In [10]:
## split dataset into train and test dataset using random_split
from torch.utils.data import random_split
train_val_size = int(0.9 * len(imageDataset))
test_size = len(imageDataset) - train_val_size
train_val_ds, test_ds = random_split(imageDataset, [train_val_size, test_size])

## split train_val_ds into train_ds and val_ds using random_split
train_size = int(0.8 * len(train_val_ds))
val_size = len(train_val_ds) - train_size

train_ds, val_ds = random_split(train_val_ds, [train_size, val_size])

print('train_val_ds: ', len(train_val_ds))
print('test_ds: ', len(test_ds))


train_ds:  18109
val_ds:  4528
test_ds:  2516


### 1. Resnes18

In [11]:
## get resnet model of image classification from torchvision
model = torchvision.models.resnet18(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

/home/ntky/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ntky/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ntky/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|█████████████████| 44.7M/44.7M [00:00<00:00, 68.2MB/s]


(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [12]:
## train model
reload(t)
from train import cross_validate
avg_lost = cross_validate(model, optimizer, loss_fn, train_val_ds, epochs=5, device=device)
print('avg_lost: ', avg_lost)
## score model
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/5, Train Loss: 0.6718, Test Loss: 0.5714, Total Time: 00 hours, 01 minutes, 36 seconds
Epoch 2/5, Train Loss: 0.4331, Test Loss: 0.3829, Total Time: 00 hours, 01 minutes, 31 seconds
Epoch 3/5, Train Loss: 0.3102, Test Loss: 0.3836, Total Time: 00 hours, 01 minutes, 25 seconds
Epoch 4/5, Train Loss: 0.2402, Test Loss: 0.3507, Total Time: 00 hours, 01 minutes, 29 seconds
Epoch 5/5, Train Loss: 0.1827, Test Loss: 0.3296, Total Time: 00 hours, 01 minutes, 24 seconds
Epoch 1/5, Train Loss: 0.1895, Test Loss: 0.1445, Total Time: 00 hours, 01 minutes, 24 seconds
Epoch 2/5, Train Loss: 0.1357, Test Loss: 0.2599, Total Time: 00 hours, 01 minutes, 24 seconds
Epoch 3/5, Train Loss: 0.1092, Test Loss: 0.1963, Total Time: 00 hours, 01 minutes, 25 seconds
Epoch 4/5, Train Loss: 0.0906, Test Loss: 0.2349, Total Time: 00 hours, 01 minutes, 24 seconds
Epoch 5/5, Train Loss: 0.0748, Test Loss: 0.2314, Total Time: 00 hours, 01 minutes, 24 seconds
Epoch 1/5, Train Loss: 0.1227, Test Loss: 0.0936, 

0.5864619879564344

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.100_epochs.small.pth')

### 1. Resnes34

In [8]:
## get resnet model of image classification from torchvision
model = torchvision.models.resnet34(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [10]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes34.small.pth')

### 2. DenseNet

In [ ]:
## get resnet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [ ]:
## train model
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

KeyboardInterrupt: 

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=100, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/10, Train Loss: 0.4070, Test Loss: 0.4468, Total Time: 00 hours, 10 minutes, 50 seconds
Epoch 2/10, Train Loss: 0.2953, Test Loss: 0.3624, Total Time: 00 hours, 12 minutes, 53 seconds
Epoch 3/10, Train Loss: 0.2196, Test Loss: 0.3772, Total Time: 00 hours, 13 minutes, 49 seconds
Epoch 4/10, Train Loss: 0.1600, Test Loss: 0.3560, Total Time: 00 hours, 16 minutes, 10 seconds
Epoch 5/10, Train Loss: 0.1331, Test Loss: 0.3690, Total Time: 00 hours, 15 minutes, 54 seconds
Epoch 6/10, Train Loss: 0.1037, Test Loss: 0.3158, Total Time: 00 hours, 15 minutes, 15 seconds
Epoch 7/10, Train Loss: 0.0790, Test Loss: 0.3453, Total Time: 00 hours, 15 minutes, 53 seconds
Epoch 8/10, Train Loss: 0.0761, Test Loss: 0.4047, Total Time: 00 hours, 15 minutes, 15 seconds
Epoch 9/10, Train Loss: 0.0618, Test Loss: 0.3804, Total Time: 00 hours, 15 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0561, Test Loss: 0.4284, Total Time: 00 hours, 16 minutes, 02 seconds


0.4667215104080109

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.100_epochs.small.pth')

In [ ]:
## get denseNet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True, num_classes=len(labels))
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [21]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/1, Train Loss: 0.6834, Test Loss: 0.7046, Total Time: 05 hours, 27 minutes, 36 seconds


0.7259557617677225

In [22]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/10, Train Loss: 0.4411, Test Loss: 0.6275, Total Time: 05 hours, 34 minutes, 19 seconds
Epoch 2/10, Train Loss: 0.3439, Test Loss: 0.6895, Total Time: 05 hours, 34 minutes, 47 seconds
Epoch 3/10, Train Loss: 0.2888, Test Loss: 0.4157, Total Time: 05 hours, 43 minutes, 16 seconds
Epoch 4/10, Train Loss: 0.2428, Test Loss: 0.4163, Total Time: 05 hours, 35 minutes, 47 seconds
Epoch 5/10, Train Loss: 0.2049, Test Loss: 0.3158, Total Time: 07 hours, 43 minutes, 43 seconds
Epoch 6/10, Train Loss: 0.1741, Test Loss: 0.5003, Total Time: 08 hours, 44 minutes, 26 seconds
Epoch 7/10, Train Loss: 0.1587, Test Loss: 0.3826, Total Time: 08 hours, 34 minutes, 52 seconds
Epoch 8/10, Train Loss: 0.1292, Test Loss: 0.3596, Total Time: 08 hours, 28 minutes, 02 seconds
Epoch 9/10, Train Loss: 0.1110, Test Loss: 0.4189, Total Time: 17 hours, 53 minutes, 29 seconds
Epoch 10/10, Train Loss: 0.1113, Test Loss: 0.3861, Total Time: 08 hours, 27 minutes, 54 seconds


0.41211051089964534

In [23]:
## save model
torch.save(model.state_dict(), 'model_densenet121.small.pth')

### 3. Using Yolov5 to extract face then using ResNet to classification the output

In [ ]:
## use yoloresnet model
import model
reload(model)
from model import YoloResnet
model = YoloResnet()
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_yoloresnet.pth')